# Prediction of drug-drug interaction using RDF2Vec

In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_curve, auc,average_precision_score
import numpy
import pandas as pd
import numpy as np

from scipy import interp
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn import metrics

import random
import numbers
import itertools

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import gc
import os

## Read RDF2Vec features

In [6]:
def getNegativeSize(all_combs, positive_pairs, n_propotion):
    x= len(all_combs) - len(positive_pairs) 
    if x < len(positive_pairs)*n_propotion:
         negative_size=x
    else:
        negative_size=len(positive_pairs)*n_propotion
        
    return negative_size

def getDataFrame(pairs, cls):
    pairs = np.array(pairs)
    if cls == 1:
        classes = np.ones(len(pairs))
    else:
        classes = np.zeros(len(pairs))
    #classes = numpy.full((len(pairs)), cls)
    data = list(zip(pairs[:,0],pairs[:,1],classes))
    #print (data)
    df = pd.DataFrame(data,columns=['Drug1','Drug2','Class'])
    return df

def select_negative_samples(train_drugs, test_drugs, train_pairs, test_pairs_C1, test_pairs_C2, test_pairs_C3, n_propotion):
    # negative samples for training
    all_combs = set(itertools.combinations(sorted(train_drugs),2))
    #print("train all pairs",len(all_combs))
    #print("train pairs", train_pairs)
    # check whether there is enough pairs to be added as negatives
    
    negative_size=getNegativeSize(all_combs, train_pairs, n_propotion)
    unknowPairs = all_combs.difference(train_pairs).difference(test_pairs_C1)
    train_negatives =random.sample(unknowPairs, negative_size)
    #print("train negatives", train_negatives)
    
    negative_size=getNegativeSize(unknowPairs, test_pairs_C1, n_propotion)
    test_negatives_C1 =random.sample(unknowPairs.difference(train_negatives), negative_size)
     
    all_combs = set([ tuple(sorted([drug1,drug2]))  for drug1 in train_drugs for drug2 in test_drugs])
    #print("C2 all pairs",len(all_combs))    
    negative_size = getNegativeSize(all_combs, test_pairs_C2, n_propotion) 
    test_negatives_C2 =random.sample(all_combs.difference(test_pairs_C2), negative_size)
    #print("test_negatives_C2 ", test_negatives_C2)
    
    all_combs = set(itertools.combinations(sorted(test_drugs),2))
    #print("C3 all pairs",len(all_combs))    
    negative_size = getNegativeSize(all_combs, test_pairs_C3, n_propotion) 
    test_negatives_C3 =random.sample(all_combs.difference(test_pairs_C3), negative_size)
    #print("test_negatives_C3 ", test_negatives_C3)
    
    return train_negatives,test_negatives_C1, test_negatives_C2, test_negatives_C3

def drugwise_k_fold_cross(all_drugs, all_pairs, n_fold ):
    n_subsets = int(len(all_drugs)/n_fold)
    subsets = dict()
    remain = all_drugs
    for i in range(0,n_fold-1):
        subsets[i] = random.sample(remain, n_subsets)
        remain =remain.difference(subsets[i])
    subsets[n_fold-1]= list(remain)
    
    for i in reversed(range(0, n_fold)):
        test_drugs = subsets[i]
        train_drugs = []
        for j in range(0, n_fold):
            if i != j:
                train_drugs.extend(subsets[j])
    
        #print("train drugs",train_drugs)
        #print("test drugs",test_drugs)
        train_pairs = []
        # training pairs
        for pair in all_pairs:
            drug1 = pair[0]
            drug2 = pair[1]
            if drug1 in train_drugs and drug2 in train_drugs:
                train_pairs.append(pair)

      
         # test pairs
        test_pairs_C1 =[]
        n = len(train_pairs)//n_fold
       
        for i in range(n):
            pair = random.choice(train_pairs)
            train_pairs.remove(pair)
            test_pairs_C1.append(pair)
        
        test_pairs_C2 =[]
        test_pairs_C3 =[]
        for pair in all_pairs:
            drug1 = pair[0]
            drug2 = pair[1]
            if drug1 in test_drugs and drug2 in test_drugs:
                test_pairs_C3.append(pair)
            elif drug1 in test_drugs or drug2 in test_drugs:
                test_pairs_C2.append(pair)

        yield train_drugs, test_drugs,train_pairs, test_pairs_C1, test_pairs_C2, test_pairs_C3        

In [7]:
def multimetric_score(estimator, X_test, y_test, scorers):
    """Return a dict of score for multimetric scoring"""
    scores = {}
    for name, scorer in scorers.items():
        if y_test is None:
            score = scorer(estimator, X_test)
        else:
            score = scorer(estimator, X_test, y_test)

        if hasattr(score, 'item'):
            try:
                # e.g. unwrap memmapped scalars
                score = score.item()
            except ValueError:
                # non-scalar?
                pass
        scores[name] = score

        if not isinstance(score, numbers.Number):
            raise ValueError("scoring must return a number, got %s (%s) "
                             "instead. (scorer=%s)"
                             % (str(score), type(score), name))
    return scores

def cross_validate(clf, all_drugs, all_pairs, embedding_df, n_fold, f, n_propotion=1):

    drug_k_fold = drugwise_k_fold_cross(all_drugs, all_pairs, n_fold)
    c1_results = pd.DataFrame()
    c2_results = pd.DataFrame()
    c3_results = pd.DataFrame()
    
    for i,(fold_data) in enumerate(drug_k_fold):
        #print (fold_data)
        train_drugs, test_drugs, train_positives, test_positives_C1, test_positives_C2, test_positives_C3 = fold_data
        print ("train drugs",len(train_drugs),"test drugs",len(test_drugs), file=f)
        train_negatives,test_negatives_C1, test_negatives_C2, test_negatives_C3 = select_negative_samples(train_drugs, test_drugs, train_positives, test_positives_C1, test_positives_C2, test_positives_C3, n_propotion=1)
        train =  pd.concat([getDataFrame(train_positives, 1),  getDataFrame(train_negatives, 0)],ignore_index=True) 
       
        test_C1 =  pd.concat([getDataFrame(test_positives_C1, 1),  getDataFrame(test_negatives_C1, 0)],ignore_index=True)
        test_C2 =  pd.concat([getDataFrame(test_positives_C2, 1),  getDataFrame(test_negatives_C2, 0)],ignore_index=True)   
        test_C3 =  pd.concat([getDataFrame(test_positives_C3, 1),  getDataFrame(test_negatives_C3, 0)],ignore_index=True) 
        train = train.merge(embedding_df, left_on='Drug1', right_on='Drug').merge(embedding_df, left_on='Drug2', right_on='Drug')
        test_C1 = test_C1.merge(embedding_df, left_on='Drug1', right_on='Drug').merge(embedding_df, left_on='Drug2', right_on='Drug')
        test_C2 = test_C2.merge(embedding_df, left_on='Drug1', right_on='Drug').merge(embedding_df, left_on='Drug2', right_on='Drug')
        test_C3 = test_C3.merge(embedding_df, left_on='Drug1', right_on='Drug').merge(embedding_df, left_on='Drug2', right_on='Drug')

        
        #train = train.merge(rdf2vec_df1, on=['Drug1']).merge(rdf2vec_df2, on=['Drug2'])
        #test_C1 = test_C1.merge(rdf2vec_df1, on=['Drug1']).merge(rdf2vec_df2, on=['Drug2'])
        #test_C2 = test_C2.merge(rdf2vec_df1, on=['Drug1']).merge(rdf2vec_df2, on=['Drug2'])
        #test_C3 = test_C3.merge(rdf2vec_df1, on=['Drug1']).merge(rdf2vec_df2, on=['Drug2'])
        #print (train.columns)
        features = train.columns.difference(['Drug1','Drug2' ,'Class', 'Drug_x', 'Drug_y'])
        X_train = train[features].values
        y_train = train['Class'].values

        X_test_C1 = test_C1[features].values
        y_test_C1 = test_C1['Class'].values
                             
        X_test_C2 =  test_C2[features].values
        y_test_C2 = test_C2['Class'].values

        X_test_C3 =  test_C3[features].values
        y_test_C3 = test_C3['Class'].values

        print("train positive set:", len(y_train[y_train==1]), " negative set:", len(y_train[y_train==0]), file=f)
        print("C1 test positive set:",len(y_test_C1[y_test_C1==1])," negative set:",len(y_test_C1[y_test_C1==0]), file=f)
        print("C2 test positive set:",len(y_test_C2[y_test_C2==1])," negative set:",len(y_test_C2[y_test_C2==0]), file=f)
        print("C3 test positive set:",len(y_test_C3[y_test_C3==1])," negative set:",len(y_test_C3[y_test_C3==0]), file=f)
        #probas_ = clf.fit(X_train, y_train).predict_proba(X_test)
        clf.fit(X_train, y_train)
        scoring = ['precision', 'recall', 'accuracy', 'roc_auc', 'f1', 'average_precision']
        scorers, multimetric = metrics.scorer._check_multimetric_scoring(clf, scoring=scoring)
        #print(scorers)
        scores = multimetric_score(clf, X_test_C1, y_test_C1, scorers)
        print ("C1",scores, file=f)
        c1_results = c1_results.append(scores, ignore_index=True)                     
                             
        scores = multimetric_score(clf, X_test_C2, y_test_C2, scorers)
        print ("C2",scores, file=f)
        c2_results = c2_results.append(scores, ignore_index=True)
        
        scores = multimetric_score(clf, X_test_C3, y_test_C3, scorers)
        print ("C3",scores, file=f)
        c3_results = c3_results.append(scores, ignore_index=True)
        
        print("-----------------------------------------------------------------------------", file=f)
        print("-----------------------------------------------------------------------------", file=f)

        del X_train, y_train, X_test_C1, y_test_C1, X_test_C2, y_test_C2, X_test_C3, y_test_C3
        del train_drugs, test_drugs, train_positives, test_positives_C1,test_positives_C2, test_positives_C3
        del train_negatives,test_negatives_C1,test_negatives_C2, test_negatives_C3
        gc.collect()
        
    return c1_results,c2_results, c3_results

In [8]:
def crossvalidation(model, commonDrugs, all_positives, embedding_df, n_fold, n_run, n_propotion=1):
    c1_results_runs = pd.DataFrame()
    c2_results_runs = pd.DataFrame()
    c3_results_runs = pd.DataFrame()
    logfile =open('log.txt', 'w')
    for i in range(n_run):
        c1_results, c2_results, c3_results = cross_validate(model, commonDrugs, all_positives, embedding_df, n_fold, logfile, n_propotion)
        c1_results_runs = c1_results_runs.append(c1_results.mean(), ignore_index=True)
        c2_results_runs = c2_results_runs.append(c2_results.mean(), ignore_index=True)
        c3_results_runs = c3_results_runs.append(c3_results.mean(), ignore_index=True)
    
    return c1_results_runs, c2_results_runs, c3_results_runs


def main(drugfeat, ddifile):

   
    print ("Processing file :",drugfeat)
    # ### Reading the RDF2Vec features
    
    embedding_df = pd.read_csv(drugfeat, delimiter='\t')

    #print('Size of the drugs that has RDF2Vec features: %d' % len(embedding_df))


    # ### Reading Drugbak v5.0 dataset
    drugbank_ddi = pd.read_csv(ddifile, delimiter='\t')


    print("Dataset size: %d" % len(drugbank_ddi)) # it should be 13892 (6946*2) but it doesn't affect the comparison procedure

    drugsInDrugbankDDI = set(drugbank_ddi['Drug1'].unique()).union(drugbank_ddi['Drug2'].unique())
    commonDrugs = drugsInDrugbankDDI.intersection(embedding_df.Drug.unique()).intersection(embedding_df.Drug.unique())

    print("Size of the drugs that appear in both DrugBank DDI dataset and drug2vec dataset: ", len(commonDrugs))

    import itertools
    pairs = []
    classes = []

    ddiKnown = set([tuple(x) for x in  drugbank_ddi[['Drug1','Drug2']].values])
            
    for comb in itertools.combinations(sorted(commonDrugs),2):
        dr1=comb[0]
        dr2=comb[1]
        if (dr1,dr2)  in ddiKnown or  (dr2,dr1)  in ddiKnown:
            pairs.append((dr1,dr2))

    all_positives = set(pairs)
    n_seed = 112
    n_fold = 10
    n_run = 10
    n_propotion=1

    # ### Naive Bayes
    # There is no hyper-parameter to tune. 
    nb_model = GaussianNB()
    print ("Naive Bayes")
    #nb_mean_fpr, nb_mean_tpr, nb_precision, nb_recall = crossvalidation(nb_model, X, y, nsplits=10) train_df_emd
    nb_scores_df = crossvalidation(nb_model, commonDrugs, all_positives, embedding_df, n_fold, n_run, n_propotion)

    # ### Logistic Regression
    # **Training with best parameters:**
    # Value for C parameter was selected as 0.01

    logistic_model = LogisticRegression(C=0.01)

    print ("Logistic Regression")
    lr_scores_df = crossvalidation(logistic_model, commonDrugs, all_positives, embedding_df, n_fold, n_run, n_propotion)


    rf_model = RandomForestClassifier(n_estimators=200, n_jobs=-1)

    print ("Random Forest")
    #rf_results_pred = None
    rf_scores_df = crossvalidation(rf_model, commonDrugs, all_positives, embedding_df, n_fold, n_run, n_propotion)

     
    return nb_scores_df, lr_scores_df, rf_scores_df

### Train with each embedding features and output the results of classifiers (NB, LR, RF)

 ### DRUGBANK DATASET

In [ ]:
foldername = 'vectors/DRUGBANK/' 
ddi_file ='data/input/ddi_v5.txt'
outfolder='Results/DRUGBANK/'

for fn in os.listdir(foldername):
    emdfile = os.path.join(foldername, fn)
    nb_results_pred, lr_results_pred, rf_results_pred = main(emdfile, ddi_file)

    nb_results_pred.to_csv(outfolder+fn[:-4]+'_nb_results_pred.csv')
    lr_results_pred.to_csv(outfolder+fn[:-4]+'_lr_results_pred.csv')
    rf_results_pred.to_csv(outfolder+fn[:-4]+'_rf_results_pred.csv')
    

 ### INTEGRATED (DRUGBANK, KEGG, PHARMGKB) DATASET

In [ ]:
foldername =  'vectors/DB_KEGG_PGK/'
fn = 'Drug2Vec_sg_200_5_5_15_2_500_uniform.txt'
ddi_file ='data/input/ddi_v5.txt'
outfolder='Results/DB_KEGG_PGK/' 

emdfile = os.path.join(foldername, fn)
nb_results_pred, lr_results_pred, rf_results_pred = main(emdfile, ddi_file)
nb_results_pred[0].to_csv(outfolder+fn[:-4]+'_nb_results_C1.csv')
nb_results_pred[1].to_csv(outfolder+fn[:-4]+'_nb_results_C2.csv')
nb_results_pred[2].to_csv(outfolder+fn[:-4]+'_nb_results_C3.csv')
lr_results_pred[0].to_csv(outfolder+fn[:-4]+'_lr_results_C1.csv')
lr_results_pred[1].to_csv(outfolder+fn[:-4]+'_lr_results_C2.csv')
lr_results_pred[2].to_csv(outfolder+fn[:-4]+'_lr_results_C3.csv')
rf_results_pred[0].to_csv(outfolder+fn[:-4]+'_rf_results_C1.csv')
rf_results_pred[1].to_csv(outfolder+fn[:-4]+'_rf_results_C2.csv')
rf_results_pred[2].to_csv(outfolder+fn[:-4]+'_rf_results_C3.csv')

Processing file : vectors/DRUGBANK/Drug2Vec_sg_200_5_5_15_2_500_uniform.txt
Dataset size: 577712
Size of the drugs that appear in both DrugBank DDI dataset and drug2vec dataset:  2124
Naive Bayes


In [ ]:
#foldername = '../rdfvec/vectors/DB_KEGG_PGK/' 
foldername =  'vectors/DB_KEGG_PGK/'
ddi_file ='data/input/ddi_v5.txt'
outfolder='Results/DB_KEGG_PGK/'
scores_runs = pd.DataFrame()

for fn in os.listdir(foldername):
    emdfile = os.path.join(foldername, fn)
    if 'trans' not in fn: continue
    nb_results_pred, lr_results_pred, rf_results_pred = main(emdfile, ddi_file)
    nb_results_pred[0].to_csv(outfolder+fn[:-4]+'_nb_results_C1.csv')
    nb_results_pred[1].to_csv(outfolder+fn[:-4]+'_nb_results_C2.csv')
    nb_results_pred[2].to_csv(outfolder+fn[:-4]+'_nb_results_C3.csv')
    lr_results_pred[0].to_csv(outfolder+fn[:-4]+'_lr_results_C1.csv')
    lr_results_pred[1].to_csv(outfolder+fn[:-4]+'_lr_results_C2.csv')
    lr_results_pred[2].to_csv(outfolder+fn[:-4]+'_lr_results_C3.csv')
    rf_results_pred[0].to_csv(outfolder+fn[:-4]+'_rf_results_C1.csv')
    rf_results_pred[1].to_csv(outfolder+fn[:-4]+'_rf_results_C2.csv')
    rf_results_pred[2].to_csv(outfolder+fn[:-4]+'_rf_results_C3.csv')
